In [17]:
import pandas as pd
import mysql.connector

In [18]:
connection = mysql.connector.connect(
    host= 'localhost',         
    user= 'root',             
    password= '123456', 
    database = 'day_48'     
)
cursor = connection.cursor()

In [3]:
cursor.execute("""
CREATE TABLE customers (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL, 
    city VARCHAR(200) NOT NULL
);
""")

In [4]:
cursor.execute("""
INSERT INTO customers (id, name, city) VALUES
(1, 'Alice', 'Hanoi'),
(2, 'Bob', 'Ho Chi Minh'),
(3, 'Charlie', 'Hanoi');
""")

connection.commit()

In [5]:
cursor.execute("""
CREATE TABLE orders (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    total INTEGER NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
);
""")

In [6]:
cursor.execute("""
INSERT INTO orders (id, customer_id, order_date, total) VALUES
(1, 1, '2024-01-10', 500),
(2, 1, '2024-03-01', 800),
(3, 2, '2024-02-20', 1200),
(4, 3, '2024-01-15', 200);
""")

connection.commit()

In [7]:
cursor.execute("""
CREATE TABLE products (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    price INTEGER NOT NULL
);
""")

In [8]:
cursor.execute("""
INSERT INTO products (id, name, price) VALUES
(1, 'Laptop', 1500),
(2, 'Mouse', 50),
(3, 'Keyboard', 100),
(4, 'Monitor', 300);
""")

connection.commit()

In [9]:
cursor.execute("""
CREATE TABLE order_items (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (product_id) REFERENCES products(id)
);
""")

In [10]:
cursor.execute("""
INSERT INTO order_items (id, order_id, product_id, quantity) VALUES
(1, 1, 2, 2),
(2, 1, 3, 1),
(3, 2, 1, 1),
(4, 2, 4, 1),
(5, 3, 1, 1);
""")

connection.commit()

In [11]:
cursor.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    department VARCHAR(200) NOT NULL
);
""")

In [13]:
cursor.execute("""
INSERT INTO employees (id, name, department) VALUES
(1, 'David', 'Sales'),
(2, 'Emma', 'Support'),
(3, 'Frank', 'Sales');
""")

connection.commit()

In [14]:
cursor.execute("""
CREATE TABLE order_assignments (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    order_id INTEGER NOT NULL,
    employee_id INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (employee_id) REFERENCES employees(id)
);
""")

In [15]:
cursor.execute("""
INSERT INTO order_assignments (id, order_id, employee_id) VALUES
(1, 1, 1),
(2, 2, 3),
(3, 3, 2);
""")

connection.commit()

### Bài 1

In [22]:
query = ("""
SELECT c.name
    , b.order_date
    , a.quantity
    , b.total
 FROM order_items a
LEFT JOIN orders b on a.order_id = b.id
LEFT JOIN customers c on b.customer_id = c.id
WHERE product_id IN (SELECT id FROM products WHERE price > 1000)
""")

data_1 = pd.read_sql_query(query, connection)
print(data_1)

    name  order_date  quantity  total
0  Alice  2024-03-01         1    800
1    Bob  2024-02-20         1   1200


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_2180\3133172740.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_3 = pd.read_sql_query(query, connection)


### Bài 2

In [23]:
query = ("""
SELECT b.name
    , SUM(a.total) total
 FROM orders a
LEFT JOIN customers b on a.customer_id = b.id
GROUP BY b.name
""")

data_2 = pd.read_sql_query(query, connection)
print(data_2)

      name   total
0    Alice  1300.0
1      Bob  1200.0
2  Charlie   200.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_2180\2421038388.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_2 = pd.read_sql_query(query, connection)


### Bài 3

In [24]:
query = ("""
SELECT a.name, a.total FROM (
SELECT b.name
    , SUM(a.total) total
 FROM orders a
LEFT JOIN customers b on a.customer_id = b.id
GROUP BY b.name
) a
ORDER BY a.total DESC
LIMIT 2
""")

data_3 = pd.read_sql_query(query, connection)
print(data_3)

    name   total
0  Alice  1300.0
1    Bob  1200.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_2180\3420196805.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_3 = pd.read_sql_query(query, connection)


### Bài 4

In [30]:
query = ("""
SELECT c.name
    , c.department
    , a.order_id
 FROM order_items a
LEFT JOIN order_assignments b on a.order_id = b.id
LEFT JOIN employees c on b.employee_id = c.id
WHERE EXISTS (SELECT 1 FROM products p WHERE p.id = a.product_id AND p.name LIKE '%Monitor%')
""")

data_4 = pd.read_sql_query(query, connection)
print(data_4)

    name department  order_id
0  Frank      Sales         2


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_2180\2305480799.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_4 = pd.read_sql_query(query, connection)


### Bài 5

In [40]:
query = ("""
SELECT b.name
    , SUM(a.total) total
 FROM orders a
LEFT JOIN customers b on a.customer_id = b.id
GROUP BY b.name
HAVING SUM(a.total) > (SUM(a.total)/COUNT(b.id))
""")

data_5 = pd.read_sql_query(query, connection)
print(data_5)

    name   total
0  Alice  1300.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_2180\1627744907.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_5 = pd.read_sql_query(query, connection)


### Bài 6

In [41]:
cursor.execute("""
CREATE TEMPORARY TABLE vip_customers (
    SELECT b.id 
        , b.name
        , SUM(a.total) total
     FROM orders a
    LEFT JOIN customers b on a.customer_id = b.id
    GROUP BY b.name, b.id
    HAVING SUM(a.total) > 1000
);
""")

query = ("""
SELECT * 
 FROM vip_customers
""")

data_6 = pd.read_sql_query(query, connection)
print(data_6)

   id   name   total
0   1  Alice  1300.0
1   2    Bob  1200.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_2180\2004924268.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_6 = pd.read_sql_query(query, connection)


### Bài 7

In [57]:
cursor.execute("""
CREATE TEMPORARY TABLE popular_products (
    SELECT b.id 
        , b.name
        , SUM(a.quantity) quantity
     FROM order_items a
    LEFT JOIN products b on a.product_id = b.id
    GROUP BY b.name, b.id
    HAVING SUM(a.quantity) >= 2
);
""")

query = ("""
SELECT * 
 FROM popular_products
""")

data_7 = pd.read_sql_query(query, connection)
print(data_7)

   id    name  quantity
0   2   Mouse       2.0
1   1  Laptop       2.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_2180\2365298657.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_7 = pd.read_sql_query(query, connection)


### Bài 8

In [71]:
query = ("""
SELECT b.name, b.id 
FROM orders a 
LEFT JOIN customers b on a.customer_id = b.id
WHERE a.id IN (
    SELECT order_id 
     FROM order_assignments 
    WHERE employee_id IN (
        SELECT employee_id 
         FROM (
            SELECT b.employee_id
            , SUM(a.total) total
             FROM orders a 
            LEFT JOIN order_assignments b on a.id = b.order_id
            GROUP BY b.employee_id
            HAVING SUM(a.total) > 1000
        ) a
    )
)
""")

data_8 = pd.read_sql_query(query, connection)
print(data_8)

  name  id
0  Bob   2


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_2180\1486653157.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_8 = pd.read_sql_query(query, connection)


### Bài 9

In [73]:
cursor.execute("""
CREATE TEMPORARY TABLE high_value_orders (
    SELECT b.id 
        , b.name
        , SUM(a.total) total
     FROM orders a
    LEFT JOIN customers b on a.customer_id = b.id
    GROUP BY b.name, b.id
    HAVING SUM(a.total) > 800
);
""")

query = ("""
SELECT name, total
 FROM high_value_orders
""")

data_9 = pd.read_sql_query(query, connection)
print(data_9)

    name   total
0  Alice  1300.0
1    Bob  1200.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_2180\1757300236.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_9 = pd.read_sql_query(query, connection)


### Bài 10

In [76]:
query = ("""
SELECT order_id, SUM(total) total
 FROM (
    SELECT a.id
        , a.order_id
        , a.quantity
        , b.price
        , a.quantity * b.price total
     FROM order_items a
    LEFT JOIN products b on a.product_id = b.id
) a
GROUP BY order_id
""")

data_10 = pd.read_sql_query(query, connection)
print(data_10)

   order_id   total
0         1   200.0
1         2  1800.0
2         3  1500.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_2180\3672412380.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_5 = pd.read_sql_query(query, connection)
